In [26]:
import pandas as pd
import os
import requests

In [27]:
raidDir   = '/Users/amcguire/Documents/PERSONAL/KOL/raidLogs/'
newRDir   = '/Users/amcguire/Library/Application Support/KoLmafia/data/test/'
outputDir = '/Users/amcguire/Documents/PERSONAL/KOL/'

multiUser = {'madowl (combined)':['themadowl','madowl','Arkcon']}

clanMap   = {'The Hogs of Exploitery':'HoE', 
             'KirbyLlama\'s Private Dungeon Clan': 'KirbyClan',
             'The Piglets of Fate': 'Piglets',
             'The Hogs of Destiny': 'HoD',
             'The 100% of Scientific FACT Club': 'FactClub',
             'Castle Greyhawk': 'Greyhawk'}

finishedUsers = ['violetinsane','Captain Scotch','kenny kamAKAzi']

runFiles  = ['r1-3389','r2-3869','r3-4283','r4-4190','r5-3503','r6-3598',
             'r7-4372','r8-3473','r9-3858','r10-3787','r11-3797','r12-3798',
             'r13-12260','r14-8393','r15-3769','r16-3496','r17-4041','r18-3582',
             'r19-4318','r20-3584','r21-3794','r22-4698','r25-3802',
             'r69-1337']
             #'r24p-1852','r23p-722','r26p-900','r27p-2770']

In [28]:
# Temporary script to convert Stary script output into old c/p format

for file in os.listdir(newRDir):
    
    try:    
        clan = file.split(sep='_')[1]
        rNum = file.split(sep='_')[2][:6]
        newFileName = 'r{1}_{0}.txt'.format(clanMap[clan],rNum)

        currFile = open('{}/{}'.format(newRDir, file)).read()

        newFile = ''

        for ct, line in enumerate(currFile.split(sep='\n')):
            try:     newFile = newFile + line.split(sep='\t')[1].replace('<b>','').replace('</b>','') + '\n'
            except:  newFile = newFile + 'LINE-ERROR' + '\n'

        print('{} file parsed. {} new lines.'.format(clan,ct))

        file = open('{}{}'.format(raidDir,newFileName),"w")

        file.write(newFile)

        file.close()

        runFiles = runFiles + [newFileName[:-4]]
    except:
        print('stupid thing fucked up')

stupid thing fucked up
The Hogs of Exploitery file parsed. 184 new lines.
KirbyLlama's Private Dungeon Clan file parsed. 189 new lines.
The Hogs of Destiny file parsed. 189 new lines.
Castle Greyhawk file parsed. 84 new lines.
The Piglets of Fate file parsed. 283 new lines.
The Hogs of Destiny file parsed. 139 new lines.
The Hogs of Exploitery file parsed. 166 new lines.
The Piglets of Fate file parsed. 125 new lines.
KirbyLlama's Private Dungeon Clan file parsed. 174 new lines.
The 100% of Scientific FACT Club file parsed. 156 new lines.
KirbyLlama's Private Dungeon Clan file parsed. 142 new lines.
The Hogs of Destiny file parsed. 0 new lines.
The Hogs of Destiny file parsed. 143 new lines.
The Hogs of Exploitery file parsed. 145 new lines.
KirbyLlama's Private Dungeon Clan file parsed. 140 new lines.
The Piglets of Fate file parsed. 182 new lines.
KirbyLlama's Private Dungeon Clan file parsed. 114 new lines.
KirbyLlama's Private Dungeon Clan file parsed. 165 new lines.
The Hogs of Ex

In [29]:
# Populate our run dictionary
allRuns = ""

for run, txtLog in enumerate(runFiles):
    # Python is zero-indexed, so we increment run by 1 here. 
    #   Also replace spaces w/ _ for playername.
    
    allRuns = allRuns+open('{}{}.txt'.format(raidDir, txtLog)).read()+"\n"

In [30]:
def collectRaiders(runs):
    """ Function to collect all dungeon raiders. Only grabs folks who
        have defeated at least one monster across all ASS runs. """
    
    raiders = []
    
    for row in runs.split('\n'):
        if ('defeated' in row) & (' (#' in row):
            name = row[:row.find(' (#')]
            raiders = list(set(raiders + [name]))
    
    return raiders
            
collectRaiders(allRuns)

['Just Eyes',
 'Stary',
 'RandomExtremity',
 'madowl',
 'John Keel',
 'The Dictator',
 'Arkcon',
 'gregmasta',
 'Whym',
 'Lyft',
 'Gausie',
 'HeroBill',
 'Volotani',
 'NukebOy1313',
 'Name Guy Man',
 'Phillammon',
 'kirByllAmA',
 'violetinsane',
 'AeNimUS',
 'playultm8',
 'threebullethamburgler',
 'ANonamUS',
 'themadowl',
 'tdog21',
 'Liora',
 'chameco',
 'Captain Scotch',
 'kenny kamAKAzi']

In [31]:
def tabulateKills(name, runs):
    """ Function to tabulate how many kills a specific individual had.
        Also outputs defeats, if we actually want that. """
    
    kills  = 0
    killed = 0
    
    for row in runs.split('\n'):
        if (name in row) & ('defeated' in row):
            if row[0:len(name)] == name: # fixing the madowl bug ;-;
                if 'was defeated' in row:
                    tSpent = int(row.split(' (')[2].split(' ')[0])
                    killed += tSpent
                else:
                    tSpent = int(row.split(' (')[2].split(' ')[0])
                    kills  += tSpent
                
    return({'kills':kills,'defeats':killed})

In [32]:
def tabulateLoots(name, runs):
    ''' Function to tabulate loot per individuals participating. Also 
        grabs skills, just so that we have em.'''
    
    loot   = []
    skills = 0
    
    for row in runs.split('\n'):
        if ('distributed' in row) and (name in row):
            distTo = row.split(' to ')[1].split(' (#')[0]
            if distTo == name:
                thisLoot = row.split(' distributed ')[1].split(' to ')[0]
                loot = loot + [thisLoot]
        elif (' used The Machine' in row) and (name in row):
            distTo = row.split(' used The ')[0].split(' (#')[0]
            if distTo == name: skills+=1
        
    # Parse out things we don't want to be considered as "true" loot
    notRealLoot = ['ghost pepper','electric Kool-Aid','skull capacitor',
                   'wriggling severed nose','Hunger™ Sauce','bottle of Bloodweiser']
    finalLoot = [x for x in loot if x not in notRealLoot]
    
    return({'loot':finalLoot,'skills':skills})
            
tabulateLoots('Captain Scotch', allRuns)

{'loot': ["Drunkula's wineglass",
  'zombie accordion',
  "Mayor Ghost's cloak",
  "Drunkula's bell",
  "Mayor Ghost's toupee",
  "Great Wolf's headband",
  "Mayor Ghost's sash",
  "Great Wolf's headband",
  "Unkillable Skeleton's restless leg",
  "Unkillable Skeleton's restless leg",
  'Quiets-Your-Steps',
  'Gets-You-Drunk',
  'HOA zombie eyes',
  'Gets-You-Drunk',
  "Drunkula's ring of haze",
  "Mayor Ghost's scissors",
  'Helps-You-Sleep'],
 'skills': 11}

In [33]:
finalDisplay = {}

# Create summary dictionary via tabulation code

for name in collectRaiders(allRuns):
    
    finalDisplay[name] = {}
    
    finalDisplay[name].update(tabulateKills(name, allRuns))
    finalDisplay[name].update(tabulateLoots(name, allRuns))
    finalDisplay[name]['finishedFlag'] = [1 if name in finishedUsers else 0][0]
    
    finalDisplay[name]['lootCount'] = len(finalDisplay[name]['loot'])

In [34]:
# Slice of code for combining duplicate users

for name in multiUser.keys():
    
    finalDisplay[name] = {}
    
    for field in ['kills','defeats','loot','skills','lootCount']:
        try:    finalDisplay[name][field] = sum([finalDisplay[x][field] for x in multiUser[name]])
        except: finalDisplay[name][field] = [].append([finalDisplay[x][field] for x in multiUser[name]])
            
    for x in multiUser[name]: finalDisplay.pop(x)

In [35]:
finalFrame = pd.DataFrame.from_dict(finalDisplay,orient='index').sort_values(by='kills',ascending=False).fillna(0)
finalFrame = finalFrame.loc[:,['kills','defeats','skills','lootCount','loot','finishedFlag']]
finalFrame['kill/skillz'] = finalFrame['kills']/(finalFrame['skills']+0.5)

finalFrame = finalFrame.sort_values(['finishedFlag','kill/skillz'], ascending=[True,False])
finalFrame

,kills,defeats,skills,lootCount,loot,finishedFlag,kill/skillz
Just Eyes,7493,92,5,7,"[Gets-You-Drunk, Drunkula's wineglass, Protect...",0.0,1362.363636
madowl (combined),14884,738,11,21,0,0.0,1294.260870
Name Guy Man,7569,7,6,17,"[Quiets-Your-Steps, Covers-Your-Head, Drunkula...",0.0,1164.461538
threebullethamburgler,6299,50,5,11,"[Drunkula's silky pants, Mayor Ghost's cloak, ...",0.0,1145.272727
Liora,548,7,0,1,[Great Wolf's left paw],0.0,1096.000000
gregmasta,13611,25,12,12,"[Gets-You-Drunk, Unkillable Skeleton's shingua...",0.0,1088.880000
Stary,6499,64,6,7,"[Mayor Ghost's scissors, Drunkula's cape, Grea...",0.0,999.846154
John Keel,4439,53,4,2,"[Unkillable Skeleton's breastplate, Unkillable...",0.0,986.444444
Whym,5368,1,5,1,[Gets-You-Drunk],0.0,976.000000
RandomExtremity,6217,16,6,7,"[Drunkula's bell, Great Wolf's rocket launcher...",0.0,956.461538


In [36]:
print("This effort has gotten {} skills for {} Loathers. Good job folks!".format(finalFrame['skills'].sum(),len(finalFrame['skills'])))

This effort has gotten 115 skills for 26 Loathers. Good job folks!


In [37]:
from datetime import date

currDate = str(date.today().strftime('%m%d'))
finalFrame.to_csv('{}dungeonLogs{}.csv'.format(outputDir,currDate))

In [13]:
n = "The Dictator"
for run in runFiles:
    print('{} - {}'.format(run,str(tabulateKills(n, open('{}{}.txt'.format(raidDir, run)).read()+"\n"))))

r1-3389 - {'kills': 0, 'defeats': 0}
r2-3869 - {'kills': 0, 'defeats': 0}
r3-4283 - {'kills': 0, 'defeats': 0}
r4-4190 - {'kills': 0, 'defeats': 0}
r5-3503 - {'kills': 0, 'defeats': 0}
r6-3598 - {'kills': 0, 'defeats': 0}
r7-4372 - {'kills': 0, 'defeats': 0}
r8-3473 - {'kills': 0, 'defeats': 0}
r9-3858 - {'kills': 0, 'defeats': 0}
r10-3787 - {'kills': 0, 'defeats': 0}
r11-3797 - {'kills': 0, 'defeats': 0}
r12-3798 - {'kills': 0, 'defeats': 0}
r13-12260 - {'kills': 0, 'defeats': 0}
r14-8393 - {'kills': 0, 'defeats': 0}
r15-3769 - {'kills': 0, 'defeats': 0}
r16-3496 - {'kills': 0, 'defeats': 0}
r17-4041 - {'kills': 0, 'defeats': 0}
r18-3582 - {'kills': 0, 'defeats': 0}
r19-4318 - {'kills': 0, 'defeats': 0}
r20-3584 - {'kills': 0, 'defeats': 0}
r21-3794 - {'kills': 0, 'defeats': 0}
r22-4698 - {'kills': 0, 'defeats': 0}
r25-3802 - {'kills': 0, 'defeats': 0}
r69-1337 - {'kills': 1103, 'defeats': 0}
r188769_HoE - {'kills': 0, 'defeats': 0}
r188757_KirbyClan - {'kills': 0, 'defeats': 0}
r1888

In [14]:
n = "Gausie"
for run in runFiles:
    print('{} - {}'.format(run,str(tabulateKills(n, open('{}{}.txt'.format(raidDir, run)).read()+"\n"))))

r1-3389 - {'kills': 0, 'defeats': 0}
r2-3869 - {'kills': 0, 'defeats': 0}
r3-4283 - {'kills': 0, 'defeats': 0}
r4-4190 - {'kills': 0, 'defeats': 0}
r5-3503 - {'kills': 0, 'defeats': 0}
r6-3598 - {'kills': 0, 'defeats': 0}
r7-4372 - {'kills': 1, 'defeats': 0}
r8-3473 - {'kills': 0, 'defeats': 0}
r9-3858 - {'kills': 1, 'defeats': 0}
r10-3787 - {'kills': 0, 'defeats': 0}
r11-3797 - {'kills': 0, 'defeats': 0}
r12-3798 - {'kills': 0, 'defeats': 1}
r13-12260 - {'kills': 9, 'defeats': 2}
r14-8393 - {'kills': 0, 'defeats': 0}
r15-3769 - {'kills': 0, 'defeats': 0}
r16-3496 - {'kills': 0, 'defeats': 0}
r17-4041 - {'kills': 0, 'defeats': 0}
r18-3582 - {'kills': 0, 'defeats': 0}
r19-4318 - {'kills': 0, 'defeats': 0}
r20-3584 - {'kills': 0, 'defeats': 0}
r21-3794 - {'kills': 0, 'defeats': 0}
r22-4698 - {'kills': 0, 'defeats': 0}
r25-3802 - {'kills': 0, 'defeats': 0}
r69-1337 - {'kills': 0, 'defeats': 0}
r188769_HoE - {'kills': 16, 'defeats': 0}
r188757_KirbyClan - {'kills': 0, 'defeats': 0}
r188819

In [15]:
import numpy as np

def roll1( x, y ):
    if ( x == 2 ) & ( y == 3 ):
        return 1
    else:
        return 0
